In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  5 16:38:38 2019

@author: jiliesio
"""

#Import Gurobi solver library
from gurobipy import *

#Data
costs=[[53,95,136,160],
[60,120,132,140],
[210, 190, 89, 71]]

capacities=[10200, 2000, 4100]
demands = [2500, 3000, 5500, 5000]

#Indexes
mills = range(3)
warehouses = range(4)

#Create the LP model in gurobi

model = Model("PP company transportation")

#Decision variables x_ij
x=[]
for i in mills:
    x.append([])
    for j in warehouses:
        x[i].append(model.addVar(lb=0, name="x%d.%d" % (i, j)))
model.update()
    

#Capacity constraints
for i in mills:
    model.addConstr(quicksum(x[i][j] for j in warehouses)<=capacities[i],"Capacity constraint")
model.update() 

#Demand constraints
for j in warehouses:
    model.addConstr(quicksum(x[i][j] for i in mills)==demands[j],"Demand constraint")
model.update() 

#objective function
model.setObjective(quicksum(costs[i][j]*x[i][j] for i in mills for j in warehouses))
model.modelSense = GRB.MINIMIZE
model.update() 

#Find optimal solution       
model.optimize()


#Collect optimal decision variable values to array x_optimal
x_optimal=[]
for i in mills:
    x_optimal.append([])
    for j in warehouses:
        x_optimal[i].append(x[i][j].x)
        
#Print optimal transportation cost and solution
print("Optimal transportation cost:")
print(model.objVal)  
print("Optimal transportation quantities:")
print(x_optimal)     


Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x215c731e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+04]
Presolve time: 0.02s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2620000e+06   6.400000e+03   0.000000e+00      0s
       3    1.5782000e+06   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.578200000e+06
Optimal transportation cost:
1578200.0
Optimal transportation quantities:
[[2500.0, 3000.0, 4400.0, 0.0], [0.0